# Sprocket Central Pty Ltd Company Customers Recommendation Project Phase #1 Data wrangling

## KPMG Virtual Internship

<img src="sprocket_central.png">

### About the Dataset:
**Sprocket Central Pty Ltd**, a medium size bikes & cycling accessories organisation which has a large dataset relating to its customers, but their team is unsure how to effectively analyse it to help optimise its marketing strategy.

The client provided us with 3 datasets:
* Customer Demographic

* Customer Addresses

* Transactions data in the past 3 months

#### Objective of the report

Our objective is to review the data quality for the three datasets to ensure that they are ready for our analysis , we will start wrangling and fixing each one of them seperately and then join them together into one dataset

In this Phase #1 we will improve the quality of Sprocket Central Pty Ltd’s data through:

* Data wrangling and Cleaning

* Features engineering

* Joinning the datasets together

### **1. Data Wrangling & cleaning**

In [1]:
#importing the necessary data analytics libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import calendar

%matplotlib inline


In [2]:
# Excel file name: 
filename = "KPMG_VI_New_raw_data_update_final.xlsx"

xls = pd.ExcelFile(filename)
sheets = xls.sheet_names
sheets

['Title Sheet',
 'Transactions',
 'NewCustomerList',
 'CustomerDemographic',
 'CustomerAddress']

### Transaction Dataset

In [3]:
# reading the Transactions sheet as trans DataFrame
Transactions = pd.read_excel(xls, sheet_name=sheets[1], skiprows=1, na_values="n/a")
Transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [5]:
Transactions.drop(axis=1, columns=Transactions.columns[13:], inplace = True)
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

### The general information about the dataframe points out to several problems:

1. list price,standard cost columns are stored as object, where in fact it should be numeric.
2. The Transactions date column should be converted into datetime columns not object
3. Transaction date column should have year,month name,day,day name columns
4. online_order,brand, product line,product class,product size,standard cost and product_first_sold_date have missing values.

## Features Engineering

In [6]:
# How many missing points in each variable
count_missing_Transactions = Transactions.isnull().sum()
percent_missing_Transactions = round(Transactions.isnull().sum()/len(Transactions) * 100, 1)
missing_Transactions = pd.concat([count_missing_Transactions, percent_missing_Transactions], axis = 1)
missing_Transactions.columns = ["Missing (count)", "Missing (%)"]
missing_Transactions

,Missing (count),Missing (%)
transaction_id,0,0.0
product_id,0,0.0
customer_id,0,0.0
transaction_date,0,0.0
online_order,360,1.8
order_status,0,0.0
brand,197,1.0
product_line,197,1.0
product_class,197,1.0
product_size,197,1.0


### How to deal with Missing Data?

#### There are many strategies to fill missing data:

1. Fill with the mean (better used in case of continous variables without outliers)
2. Fill with the median (better used in case of continous variables with outliers)
3. Fill with the mode (better used in case of categorical variables)
4. Drop the entire variable "attribute" if the number of missing points is too large
5. Drop the rows containing null values if it's not going to affect our analysis

#### Based on the above strategies:

* online order: has just 1.8% missing values, se we will fill them with the mode (the most frequent data point)
* brand, product line,product class,product size,standard cost and product_first_sold_date have 1% missing values , they seem to have the same missing data pattern, we will remove the rows contatining their null values since it is just 1% and will not affect our analysis

In [7]:
# filling the online order column with the mode value 
Transactions['online_order'] = Transactions['online_order'].fillna(Transactions['online_order'].mode()[0])

In [8]:
# Removing other data rows with nul values
Transactions.dropna(axis=0,inplace=True)

In [9]:
# checking results after removing null values
Transactions.isnull().sum().sum()

0

In [10]:
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  int64         
 1   product_id               19803 non-null  int64         
 2   customer_id              19803 non-null  int64         
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19803 non-null  float64       
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [11]:
Transactions.duplicated().sum()

0

In [12]:
# Collecting the categorical columns into  list
cat_col=[x for x in Transactions.dtypes.index if Transactions.dtypes[x]=='object']
cat_col

['order_status', 'brand', 'product_line', 'product_class', 'product_size']

In [13]:
#checking for duplicated values in the categorical columns nd the accuracy of the values
for col in cat_col:
    print(col)
    print(Transactions[col].unique())
    print('\n*******\n')

order_status
['Approved' 'Cancelled']

*******

brand
['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B']

*******

product_line
['Standard' 'Road' 'Mountain' 'Touring']

*******

product_class
['medium' 'low' 'high']

*******

product_size
['medium' 'large' 'small']

*******



In [14]:
Transactions.online_order = Transactions.online_order.astype(int)

In [15]:
#change product_first_sold_date
Transactions.product_first_sold_date = Transactions.product_first_sold_date.map(
    lambda t: dt.datetime.fromordinal(dt.datetime(1900, 1, 1).toordinal() + int(t) - 2))

#### We will create new variables as follows:

* Transaction year
* Transaction month name
* Transaction day
* Transaction day name

In [16]:
# Changing transactions date column into transaction year,month,day,day_name columns
Transactions['Transaction_year']=Transactions['transaction_date'].dt.year
Transactions['Transaction_month']=Transactions['transaction_date'].dt.month_name()
Transactions['Transaction_day']=Transactions['transaction_date'].dt.day
Transactions['day_of_the_week']=Transactions['transaction_date'].dt.day_name()

# converting the transaction year , day columnes into string columns
Transactions['Transaction_year']=Transactions['Transaction_year'].astype(str)
Transactions['Transaction_day']=Transactions['Transaction_day'].astype(str)

In [17]:
Transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  int64         
 1   product_id               19803 non-null  int64         
 2   customer_id              19803 non-null  int64         
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19803 non-null  int32         
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

**Transactions dataset is clean and ready for analysis**


### CustomerDemographic dataset sheet:

In [18]:
# reading the NewCustomerList sheet as trans DataFrame
CDdf = pd.read_excel(xls, sheet_name=sheets[3], skiprows=1, na_values="n/a")
CDdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [19]:
#drop undefined datafields and first and last names.
CDdf.drop(axis=1, columns=CDdf.columns[13:], inplace = True)
CDdf.drop(axis=1, columns=['first_name','last_name','default'], inplace=True)
CDdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   gender                               4000 non-null   object        
 2   past_3_years_bike_related_purchases  4000 non-null   int64         
 3   DOB                                  3913 non-null   datetime64[ns]
 4   job_title                            3494 non-null   object        
 5   job_industry_category                3344 non-null   object        
 6   wealth_segment                       4000 non-null   object        
 7   deceased_indicator                   4000 non-null   object        
 8   owns_car                             4000 non-null   object        
 9   tenure                               3913 non-null   float64       
dtypes: datetime6

#### The general information about the dataframe points out to several problems:

1. **past_3_years_bike_related_purchases**,**tenure** is stored as object, where in fact it has no numeric meaning.
2. **last_name,DOB,job_title,job_industry_category,default,tenure** has null values


### Features Engineering

In [20]:
# How many missing points in each variable
count_missing_Customer_demographic = CDdf.isnull().sum()
percent_missing_Customer_demographic = round(CDdf.isnull().sum()/len(CDdf) * 100, 1)
missing_train = pd.concat([count_missing_Customer_demographic, percent_missing_Customer_demographic], axis = 1)
missing_train.columns = ["Missing (count)", "Missing (%)"]
missing_train

,Missing (count),Missing (%)
customer_id,0,0.0
gender,0,0.0
past_3_years_bike_related_purchases,0,0.0
DOB,87,2.2
job_title,506,12.6
job_industry_category,656,16.4
wealth_segment,0,0.0
deceased_indicator,0,0.0
owns_car,0,0.0
tenure,87,2.2


#### Dealing with Missing Data

all of the categorical columns will either be filling the null values with the mode or dropping the column entirley if the missing percentage is high.

1. DOB: has 2.2% so we fill nulls with mean.
2. job_title: has 12.6% we will fill these values with random choices of the 12.5% most repeated values of the feautre.
3. job_industry_category: has 16.4% so we fill nulls with random choices of the unique values of the feautre.
4. Tenure: has 2.2% missing from the column so we will fill it with mean.

In [21]:
#Filling in columns with mode 
CDdf['DOB'] = CDdf['DOB'].fillna(CDdf['DOB'].mode()[0])
CDdf['tenure'] = CDdf['tenure'].fillna(CDdf['tenure'].mean())

In [22]:
#filling the null values in Job title with the 12.5% most repeated Job_titles to avoid any missleading skewed data.
n_p = int(len(CDdf.job_title.value_counts())*12.5/100)
CDdf.job_title.value_counts()[:n_p+1]
CDdf['job_title'] = CDdf['job_title'].fillna(np.random.choice(np.array(CDdf.job_title.value_counts()[:10].index)))

In [23]:
#filling the null values with random choices of the unique values in the feature based on the propability of occurance of each value
v_list = CDdf.job_industry_category.value_counts()
p_list = [value/v_list.sum() for value in v_list]
CDdf['job_industry_category'] = CDdf['job_industry_category'].fillna(np.random.choice(np.array(v_list.index), p=p_list))

In [24]:
# changing tenure datatype into int
CDdf.tenure = CDdf.tenure.astype(int)

In [25]:
CDdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   gender                               4000 non-null   object        
 2   past_3_years_bike_related_purchases  4000 non-null   int64         
 3   DOB                                  4000 non-null   datetime64[ns]
 4   job_title                            4000 non-null   object        
 5   job_industry_category                4000 non-null   object        
 6   wealth_segment                       4000 non-null   object        
 7   deceased_indicator                   4000 non-null   object        
 8   owns_car                             4000 non-null   object        
 9   tenure                               4000 non-null   int32         
dtypes: datetime6

There's an unexpected Date of Birth on the 'DOB' column....

--> Change that date into the most reasonable close version that could have been occured due to data entry error.

In [26]:
CDdf.DOB.min()

Timestamp('1843-12-21 00:00:00')

In [27]:
#extracting the index of the record with the faulty DOB
CDdf[CDdf.DOB==_]

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
33,34,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,20


In [28]:
#change '1843' into '1943'
CDdf.loc[[33],['DOB']] = pd.Timestamp(1943,12,21)

In [29]:
# Collecting the categorical columns into  list
cat_col=[x for x in CDdf.dtypes.index if CDdf.dtypes[x]=='object']
cat_col

['gender',
 'job_title',
 'job_industry_category',
 'wealth_segment',
 'deceased_indicator',
 'owns_car']

In [30]:
#checking for duplicated values in the categorical columns nd the accuracy of the values
for col in cat_col:
    print(col)
    print(CDdf[col].unique())
    print('\n*******\n')

gender
['F' 'Male' 'Female' 'U' 'Femal' 'M']

*******

job_title
['Executive Secretary' 'Administrative Officer' 'Recruiting Manager'
 'Desktop Support Technician' 'Senior Editor' 'Media Manager I'
 'Business Systems Development Analyst' 'Senior Quality Engineer'
 'Nuclear Power Engineer' 'Developer I' 'Account Executive'
 'Junior Executive' 'Media Manager IV' 'Sales Associate' 'Professor'
 'Geological Engineer' 'Project Manager' 'Safety Technician I'
 'Research Assistant I' 'Accounting Assistant III' 'Editor'
 'Research Nurse' 'Safety Technician III' 'Staff Accountant III'
 'Legal Assistant' 'Product Engineer' 'Information Systems Manager'
 'VP Quality Control' 'Social Worker' 'Senior Cost Accountant'
 'Assistant Media Planner' 'Payment Adjustment Coordinator' 'Food Chemist'
 'Accountant III' 'Director of Sales' 'Senior Financial Analyst'
 'Registered Nurse' 'Biostatistician II' 'Computer Systems Analyst II'
 'Software Test Engineer II' 'Paralegal' 'VP Sales'
 'Chief Design Engineer' 

#### Data Consistency Issues:
1. **gender** has F, Female, and Femal indicating 'Female' and 'M' and 'Male indicating 'Male' change both into only Male and Female.
2. **deceased_indicator** has N and Y change them into No and Yes to maintain consistency of expression across attributes.

In [31]:
#replacing all female representations into only 'Female' and all male representations into 'Male'
CDdf.gender.replace(to_replace=['Femal', 'F'], value='Female', inplace=True)
CDdf.gender.replace(to_replace='M', value='Male', inplace=True) 

In [32]:
CDdf.gender.unique()

array(['Female', 'Male', 'U'], dtype=object)

**Dealing with 'U' in gender column** 

---> replace it with random choice between male and Female depending on the propotion of each in the data.

In [33]:
valid_values = CDdf[~(CDdf.gender=='U')].gender.value_counts()
CDdf.gender.replace(to_replace='U', value=np.random.choice(np.array(valid_values.index), p=(valid_values/valid_values.sum())), inplace=True)

In [34]:
#chang deceased_indicator values fron Y and N to Yes and No respectively
CDdf.deceased_indicator.replace(to_replace='Y', value="Yes", inplace=True)
CDdf.deceased_indicator.replace(to_replace='N', value="No", inplace=True)

* **Change DOB column into Age column for better use in Modeling process**
* drop DOB column afterwards.

In [35]:
#creating Age column from DOB columns
today = dt.date.today()
CDdf['Age'] = CDdf.DOB.apply(lambda d: today.year - d.year - ((today.month, today.day) < (d.month, d.day)))

In [36]:
#dropping DOB column
CDdf.drop(axis=1, columns='DOB', inplace=True)

In [37]:
CDdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   customer_id                          4000 non-null   int64 
 1   gender                               4000 non-null   object
 2   past_3_years_bike_related_purchases  4000 non-null   int64 
 3   job_title                            4000 non-null   object
 4   job_industry_category                4000 non-null   object
 5   wealth_segment                       4000 non-null   object
 6   deceased_indicator                   4000 non-null   object
 7   owns_car                             4000 non-null   object
 8   tenure                               4000 non-null   int32 
 9   Age                                  4000 non-null   int64 
dtypes: int32(1), int64(3), object(6)
memory usage: 203.2+ KB


In [38]:
CDdf.head()

,customer_id,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Female,93,Executive Secretary,Health,Mass Customer,No,Yes,11,68
1,2,Male,81,Administrative Officer,Financial Services,Mass Customer,No,Yes,16,41
2,3,Male,61,Recruiting Manager,Property,Mass Customer,No,Yes,15,68
3,4,Male,33,Desktop Support Technician,IT,Mass Customer,No,No,7,60
4,5,Female,56,Senior Editor,Telecommunications,Affluent Customer,No,Yes,8,44


#### Customer demographic dataset is clean and ready for analysis

## Customer address dataset sheet:

In [39]:
# reading the NewCustomerList sheet as trans DataFrame
CAdf = pd.read_excel(xls, sheet_name=sheets[4], skiprows=1, na_values="n/a")
CAdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         3999 non-null   int64  
 1   address             3999 non-null   object 
 2   postcode            3999 non-null   int64  
 3   state               3999 non-null   object 
 4   country             3999 non-null   object 
 5   property_valuation  3999 non-null   int64  
 6   Unnamed: 6          0 non-null      float64
 7   Unnamed: 7          0 non-null      float64
 8   Unnamed: 8          0 non-null      float64
 9   Unnamed: 9          0 non-null      float64
 10  Unnamed: 10         0 non-null      float64
 11  Unnamed: 11         0 non-null      float64
 12  Unnamed: 12         0 non-null      float64
 13  Unnamed: 13         0 non-null      float64
 14  Unnamed: 14         0 non-null      float64
 15  Unnamed: 15         0 non-null      float64
 16  Unname

In [40]:
CAdf.drop(axis=1, columns=CAdf.columns[6:], inplace = True)
CAdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 140.7+ KB


In [41]:
CAdf.duplicated().sum()

0

In [42]:
# Collecting the categorical columns into  list
cat_col=[x for x in CAdf.dtypes.index if CAdf.dtypes[x]=='object']
cat_col

['address', 'state', 'country']

In [43]:
#checking for duplicated values in the categorical columns nd the accuracy of the values
for col in cat_col:
    print(col)
    print(CAdf[col].unique())
    print('\n*******\n')

address
['060 Morning Avenue' '6 Meadow Vale Court' '0 Holy Cross Court' ...
 '87 Crescent Oaks Alley' '8194 Lien Street' '320 Acker Drive']

*******

state
['New South Wales' 'QLD' 'VIC' 'NSW' 'Victoria']

*******

country
['Australia']

*******



#### Cleaning String data columns:
1. **address** column data won't be effective at the modeling stage ---> drop it.
2. **state** has in consistent values ---> change 'Victoria' into 'VIC' and 'New South wales' into 'NSW'
3. **country** has one value only to all custoners ---> dropit.

In [44]:
##Dropping address and country columns
CAdf.drop(axis=1, columns=['address', 'country'], inplace=True)

In [45]:
#Change 'Victoria' into 'VIC' and 'New South Wales' into 'NSW'
CAdf.replace({'state':{'Victoria':'VIC','New South Wales':'NSW'}},inplace=True)

In [46]:
CAdf.state.unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

#### Customer Address data set is clean and ready for analysis

## Merge both Customer Demographics master and Customer Address data sets:

In [47]:
## merge both CDdf and CAdf on ' cutomer_id' as 'outer' join.
Cdf = pd.merge(CDdf, CAdf, how='outer', on='customer_id', indicator=True)
Cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4003 entries, 0 to 4002
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   customer_id                          4003 non-null   int64   
 1   gender                               4000 non-null   object  
 2   past_3_years_bike_related_purchases  4000 non-null   float64 
 3   job_title                            4000 non-null   object  
 4   job_industry_category                4000 non-null   object  
 5   wealth_segment                       4000 non-null   object  
 6   deceased_indicator                   4000 non-null   object  
 7   owns_car                             4000 non-null   object  
 8   tenure                               4000 non-null   float64 
 9   Age                                  4000 non-null   float64 
 10  postcode                             3999 non-null   float64 
 11  state            

In [48]:
Cdf[Cdf._merge!='both']

,customer_id,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,postcode,state,property_valuation,_merge
2,3,Male,61.0,Recruiting Manager,Property,Mass Customer,No,Yes,15.0,68.0,NaN,NaN,NaN,left_only
9,10,Female,49.0,Senior Quality Engineer,Financial Services,Mass Customer,No,Yes,20.0,33.0,NaN,NaN,NaN,left_only
21,22,Female,79.0,Desktop Support Technician,IT,Mass Customer,No,No,11.0,59.0,NaN,NaN,NaN,left_only
22,23,Male,43.0,Desktop Support Technician,Telecommunications,High Net Worth,No,Yes,1.0,27.0,NaN,NaN,NaN,left_only
4000,4001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2756.0,NSW,10.0,right_only
4001,4002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4032.0,QLD,7.0,right_only
4002,4003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2251.0,NSW,7.0,right_only


**There're 4 customer Ids on Customer Demographics has no Customer Address data and 3 Customer Ids in customer Address data has no records in Customer Dmographics data.**

**----> to preserve completness of the data, I'll drop all 7 records.**

In [49]:
#dropiing records with _merge column != 'both' and dropping _merge column.
Cdf.dropna(inplace=True)
Cdf.drop(axis=1, columns='_merge', inplace=True)

In [50]:
Cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3996 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          3996 non-null   int64  
 1   gender                               3996 non-null   object 
 2   past_3_years_bike_related_purchases  3996 non-null   float64
 3   job_title                            3996 non-null   object 
 4   job_industry_category                3996 non-null   object 
 5   wealth_segment                       3996 non-null   object 
 6   deceased_indicator                   3996 non-null   object 
 7   owns_car                             3996 non-null   object 
 8   tenure                               3996 non-null   float64
 9   Age                                  3996 non-null   float64
 10  postcode                             3996 non-null   float64
 11  state                         

In [51]:
#changing all object data type columns into category.
cat_col=[x for x in Cdf.dtypes.index if Cdf.dtypes[x]=='object']
#checking for duplicated values in the categorical columns nd the accuracy of the values
for col in cat_col:
    Cdf[col] = Cdf[col].astype('category')
Cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3996 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   customer_id                          3996 non-null   int64   
 1   gender                               3996 non-null   category
 2   past_3_years_bike_related_purchases  3996 non-null   float64 
 3   job_title                            3996 non-null   category
 4   job_industry_category                3996 non-null   category
 5   wealth_segment                       3996 non-null   category
 6   deceased_indicator                   3996 non-null   category
 7   owns_car                             3996 non-null   category
 8   tenure                               3996 non-null   float64 
 9   Age                                  3996 non-null   float64 
 10  postcode                             3996 non-null   float64 
 11  state            

#### Customer Data is now clean and ready for analysis

## Merge Transactions data with Customer data using left join:

In [52]:
#merge Transcations dataframe and Cdf with merge using left join both on customer_id
data = pd.merge(Transactions, Cdf, how='left', on='customer_id', indicator=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 0 to 19802
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   transaction_id                       19803 non-null  int64         
 1   product_id                           19803 non-null  int64         
 2   customer_id                          19803 non-null  int64         
 3   transaction_date                     19803 non-null  datetime64[ns]
 4   online_order                         19803 non-null  int32         
 5   order_status                         19803 non-null  object        
 6   brand                                19803 non-null  object        
 7   product_line                         19803 non-null  object        
 8   product_class                        19803 non-null  object        
 9   product_size                         19803 non-null  object        
 10  list_price

In [53]:
data[data._merge!='both']

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,postcode,state,property_valuation,_merge
117,118,20,22,2017-03-21,0,Approved,Trek Bicycles,Standard,medium,small,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3458,3480,32,22,2017-09-18,0,Approved,Giant Bicycles,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3482,3504,38,22,2017-07-16,1,Approved,Solex,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4533,4565,5,10,2017-08-26,0,Approved,Trek Bicycles,Mountain,low,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4569,4601,65,22,2017-05-29,1,Approved,WeareA2B,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5555,5598,85,23,2017-12-20,0,Approved,WeareA2B,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5911,5956,21,10,2017-06-20,1,Approved,Solex,Standard,medium,large,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7365,7429,54,23,2017-10-06,0,Approved,WeareA2B,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7880,7951,19,23,2017-08-03,1,Approved,Trek Bicycles,Mountain,low,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8135,8207,53,22,2017-09-19,0,Approved,OHM Cycles,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


**There're 30 with customer Ids non exsistent in the Customer dataset,**
1. The group of 4 customers dropped while cleaning Customer dataframe.
2. and customer_id has never been on either Customer Demographics data or on Customer Address data.

**---> Drop these 30 records**

In [54]:
#Dropping records with NaN values in data dataframe
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19773 entries, 0 to 19802
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   transaction_id                       19773 non-null  int64         
 1   product_id                           19773 non-null  int64         
 2   customer_id                          19773 non-null  int64         
 3   transaction_date                     19773 non-null  datetime64[ns]
 4   online_order                         19773 non-null  int32         
 5   order_status                         19773 non-null  object        
 6   brand                                19773 non-null  object        
 7   product_line                         19773 non-null  object        
 8   product_class                        19773 non-null  object        
 9   product_size                         19773 non-null  object        
 10  list_price

### Exporting the data Dataframe into 'customer_data_wrangling.csv'

In [55]:
data.to_csv('customer_data_wrangling.csv', index=False)